In [1]:
import numpy as np
import pandas as pd
import torch
import pyro
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import pyro.distributions as dist
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import torch.nn as nn
# For reproducibility
np.random.seed(0)
torch.manual_seed(0)
pyro.set_rng_seed(0)


# M6: MLP-based clustering model

In [2]:
import pandas as pd

# Define the path to your CSV file
file_path = "cleaned_hurricane_damage_data.csv"

# Load the CSV file
df = pd.read_csv(file_path)

# Display the first 5 rows
print(df.head())


   B01001_001E  Household Income Distribution  Median Household Income  \
0       1675.0                          860.0                  75444.0   
1       2221.0                         1070.0                 140313.0   
2       1904.0                         1098.0                  83750.0   
3       1147.0                          517.0                  62054.0   
4       4946.0                         3231.0                  29737.0   

   B19001_002E  B19001_003E  B19001_004E  B19001_005E  B19001_006E  \
0         23.0         98.0         21.0         10.0         52.0   
1         45.0         24.0         16.0         17.0          0.0   
2         72.0         76.0         50.0         72.0          0.0   
3          0.0          0.0          0.0         60.0         61.0   
4        373.0        471.0        391.0        176.0        217.0   

   B19001_007E  B19001_008E  ...  Longitude  storm_county    ppt_mean  \
0         10.0          9.0  ...   -80.3117  201007_12086  11

/var/folders/p3/9thmbny930lcn8vcx6g216br0000gp/T/ipykernel_95037/4225642204.py:7: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ─────────────────────────── Column Mapping ───────────────────────────
census_svi_column_mapping = {
    'B01001_001E': 'Total Population',
    'B19001_001E': 'Household Income Distribution',
    'B19013_001E': 'Median Household Income',
    'B19001_002E': 'Income Bracket 1',
    'B19001_003E': 'Income Bracket 2',
    'B19001_004E': 'Income Bracket 3',
    'B19001_005E': 'Income Bracket 4',
    'B19001_006E': 'Income Bracket 5',
    'B19001_007E': 'Income Bracket 6',
    'B19001_008E': 'Income Bracket 7',
    'B19001_009E': 'Income Bracket 8',
    'B19001_010E': 'Income Bracket 9',
    'B19001_011E': 'Income Bracket 10',
    'B19001_012E': 'Income Bracket 11',
    'B19001_013E': 'Income Bracket 12',
    'B19001_014E': 'Income Bracket 13',
    'B19001_015E': 'Income Bracket 14',
    'B19001_016E': 'Income Bracket 15',
    'B19001_017E': 'Income Bracket 16',
    'state': 'State',
    'county': 'County',
    'tract': 'Census Tract',
    'year': 'Year',
    'STATE_COUNTY_FIPS': 'State-County FIPS Code',
    'GEOID': 'Geographic Identifier',
    'FIPS': 'FIPS Code',
    'RPL_THEMES': 'SVI Themes',
    'CZ_FIPS': 'Combined Statistical Area FIPS',
    'ST': 'State Abbreviation',
    'LOCATION': 'Location',
    'E_TOTPOP': 'Estimated Total Population',
    'M_TOTPOP': 'Margin of Error Total Population',
    'E_HU': 'Estimated Housing Units',
    'M_HU': 'Margin of Error Housing Units',
    'E_UNEMP': 'Estimated Unemployed',
    'M_UNEMP': 'Margin of Error Unemployed',
    'E_LIMENG': 'Estimated Limited English Proficiency',
    'M_LIMENG': 'Margin of Error Limited English Proficiency',
    'E_MUNIT': 'Estimated Multi-Unit Housing',
    'M_MUNIT': 'Margin of Error Multi-Unit Housing',
    'E_MOBILE': 'Estimated Mobile Homes',
    'M_MOBILE': 'Margin of Error Mobile Homes',
    'E_CROWD': 'Estimated Crowded Housing',
    'M_CROWD': 'Margin of Error Crowded Housing',
    'E_NOVEH': 'Estimated No Vehicle',
    'M_NOVEH': 'Margin of Error No Vehicle',
    'DAMAGE_PROPERTY': 'Damage to Property',
    'DAMAGE_CROPS': 'Damage to Crops',
    'BEGIN_YEARMONTH': 'Begin Year-Month',
    'LAT': 'Latitude',
    'LON': 'Longitude',
    'storm_county': 'Storm County'
}


# ─────────────────────────── Feature Engineering ───────────────────────────
def add_features(df):
    df = df.copy()
    eps = 1e-6

    df['pct_unemp']  = df['Estimated Unemployed'] / (df['Total Population'] + eps)
    df['pct_limm']   = df['Estimated Limited English Proficiency'] / (df['Total Population'] + eps)
    df['pct_noveh']  = df['Estimated No Vehicle'] / (df['Total Population'] + eps)
    df['pct_mobile'] = df['Estimated Mobile Homes'] / (df['Estimated Housing Units'] + eps)
    df['pct_crowd']  = df['Estimated Crowded Housing'] / (df['Estimated Housing Units'] + eps)

    lows  = ['Income Bracket 1', 'Income Bracket 2', 'Income Bracket 3']
    highs = ['Income Bracket 14', 'Income Bracket 15', 'Income Bracket 16']
    df['low_inc_pct']  = df[lows].sum(axis=1) / (df['Household Income Distribution'] + eps)
    df['high_inc_pct'] = df[highs].sum(axis=1) / (df['Household Income Distribution'] + eps)
    df['inc_ineq']     = df['high_inc_pct'] / (df['low_inc_pct'] + eps)

    df['log_pop']    = np.log1p(df['Total Population'])
    df['log_medinc'] = np.log1p(df['Median Household Income'])
    df['pop_poverty'] = df['Total Population'] * df['low_inc_pct']

    df['Begin Year-Month'] = pd.to_datetime(df['Begin Year-Month'], format='%Y%m')
    df['month'] = df['Begin Year-Month'].dt.month

    return df

# Define your feature columns
features = [
    'log_pop', 'log_medinc',
    'pct_unemp', 'pct_limm', 'pct_noveh',
    'low_inc_pct', 'pop_poverty', 'high_inc_pct',
    'ppt_mean', 'tmean_mean'
]


# ─────────────────────────── Main Pipeline ───────────────────────────
# Load and rename
full = pd.read_csv("cleaned_hurricane_damage_data.csv")
full = full.rename(columns=census_svi_column_mapping)
full = full.loc[:, ~full.columns.duplicated()]

# Remove rows with no damage
full = full[full['Damage to Property'] != 0].dropna(subset=['Damage to Property'])

# Add derived features
full = add_features(full)

# Group target (sum) and features (mean)
y_group = full.groupby(['Year', 'State-County FIPS Code'])[['Damage to Property']].sum().reset_index()
x_group = full.groupby(['Year', 'State-County FIPS Code'])[features + ['Latitude', 'Longitude']].mean().reset_index()

# Merge target and predictors
full_data = y_group.merge(x_group, on=['Year', 'State-County FIPS Code'])

# ─────────────────────────── Train-Test Split ───────────────────────────
train_full = full_data[full_data['Year'] < 2020].reset_index(drop=True)
test_full  = full_data[full_data['Year'] == 2020].reset_index(drop=True)

# ─────────────────────────── Spatial Clustering ───────────────────────────
coords = train_full[['Latitude', 'Longitude']].values
kmeans = KMeans(n_clusters=10, random_state=0).fit(coords)
train_full['spatial_cluster'] = kmeans.labels_

print(f"Train size: {train_full.shape[0]} | Test size: {test_full.shape[0]}")


Train size: 746 | Test size: 199


/var/folders/p3/9thmbny930lcn8vcx6g216br0000gp/T/ipykernel_95037/1913198509.py:115: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv("cleaned_hurricane_damage_data.csv")


In [5]:
def prepare_Xy(df):
    df = df.dropna(subset=features + ['Damage to Property'])
    X = df[features].values
    y = np.log1p(df['Damage to Property'].values)
    return X, y
    
def model(X, y=None):
    n = X.shape[1]
    intercept = pyro.sample("intercept", dist.Normal(0., 10.))
    weights   = pyro.sample("weights", dist.Normal(torch.zeros(n), 10*torch.ones(n)).to_event(1))
    sigma     = pyro.sample("sigma", dist.HalfNormal(10.))
    mu = intercept + (X * weights).sum(-1)
    with pyro.plate("data", X.shape[0]):
        pyro.sample("obs", dist.Normal(mu, sigma), obs=y)

def guide(X, y=None):
    n = X.shape[1]
    pyro.sample("intercept",
        dist.Normal(pyro.param("intercept_loc", torch.tensor(0.)),
                    pyro.param("intercept_scale", torch.tensor(1.), constraint=dist.constraints.positive)))
    pyro.sample("weights",
        dist.Normal(pyro.param("weights_loc", torch.zeros(n)),
                    pyro.param("weights_scale", torch.ones(n), constraint=dist.constraints.positive)).to_event(1))
    pyro.sample("sigma",
        dist.HalfNormal(pyro.param("sigma_loc", torch.tensor(1.), constraint=dist.constraints.positive)))

def safe_metrics(y_true, y_pred):
    mask = (
        (~np.isnan(y_true)) & (~np.isnan(y_pred)) &
        (~np.isinf(y_true)) & (~np.isinf(y_pred))
    )
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    return mae, rmse

# Main Loop

In [6]:
import torch
import torch.nn as nn
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Utility: Compute MAE and RMSE
def safe_metrics(y_true, y_pred):
    mask = (
        (~np.isnan(y_true)) & (~np.isnan(y_pred)) &
        (~np.isinf(y_true)) & (~np.isinf(y_pred))
    )
    y_true_clean = y_true[mask]
    y_pred_clean = y_pred[mask]
    mae = mean_absolute_error(y_true_clean, y_pred_clean)
    rmse = np.sqrt(mean_squared_error(y_true_clean, y_pred_clean))
    return mae, rmse

# MLP-based clustering model
class InnerClusteringMLP(nn.Module):
    def __init__(self, input_dim, latent_dim=2, out_clusters=4):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, latent_dim)
        )
        self.cluster_head = nn.Linear(latent_dim, out_clusters)

    def forward(self, x):
        latent = self.encoder(x)
        logits = self.cluster_head(latent)
        return logits

def nested_spatial_cv_with_mlp(train_data, features, outer_k=8, inner_k=4, n_epochs=2000):
    outer_coords = train_data[['Latitude', 'Longitude']].values
    outer_kmeans = KMeans(n_clusters=outer_k, random_state=0).fit(outer_coords)
    train_data['outer_cluster'] = outer_kmeans.labels_

    outer_results = []
    all_error_dfs = []

    for outer_fold in range(outer_k):
        print(f"\n=== OUTER Fold {outer_fold+1}/{outer_k} ===")
        outer_val = train_data[train_data['outer_cluster'] == outer_fold]
        outer_train = train_data[train_data['outer_cluster'] != outer_fold].copy()

        # MLP clustering
        mlp_features = outer_train[features].fillna(0).values
        scaler_mlp = StandardScaler().fit(mlp_features)
        X_mlp = torch.tensor(scaler_mlp.transform(mlp_features), dtype=torch.float32)
        mlp = InnerClusteringMLP(input_dim=X_mlp.shape[1], latent_dim=2, out_clusters=inner_k)
        mlp.eval()

        with torch.no_grad():
            inner_logits = mlp(X_mlp)
            inner_cluster_ids = torch.argmax(inner_logits, dim=1).numpy()

        outer_train.loc[:, 'inner_cluster'] = inner_cluster_ids

        best_model_params = None
        best_val_loss = float('inf')

        for inner_fold in range(inner_k):
            print(f"\n--- INNER Fold {inner_fold+1}/{inner_k} ---")
            inner_val = outer_train[outer_train['inner_cluster'] == inner_fold]
            inner_train = outer_train[outer_train['inner_cluster'] != inner_fold]

            if inner_val.empty or inner_train.empty:
                print(f"Skipping empty inner fold {inner_fold}")
                continue

            X_train = inner_train[features].values
            y_train = np.log1p(inner_train['Damage to Property'].values)
            X_val = inner_val[features].values
            y_val = np.log1p(inner_val['Damage to Property'].values)

            scaler = StandardScaler().fit(X_train)
            X_train_t = torch.tensor(scaler.transform(X_train), dtype=torch.float32)
            X_val_t = torch.tensor(scaler.transform(X_val), dtype=torch.float32)
            y_train_t = torch.tensor(y_train, dtype=torch.float32)
            y_val_t = torch.tensor(y_val, dtype=torch.float32)

            def model(X, y=None):
                n_features = X.shape[1]
                intercept = pyro.sample("intercept", dist.Normal(0., 10.))
                weights = pyro.sample("weights", dist.Normal(torch.zeros(n_features), 10 * torch.ones(n_features)).to_event(1))
                sigma = pyro.sample("sigma", dist.HalfNormal(10.))
                mu = intercept + (X * weights).sum(-1)
                with pyro.plate("data", X.shape[0]):
                    pyro.sample("obs", dist.Normal(mu, sigma), obs=y)

            def guide(X, y=None):
                n_features = X.shape[1]
                pyro.sample("intercept", dist.Normal(pyro.param("intercept_loc", torch.tensor(0.0)),
                                                     pyro.param("intercept_scale", torch.tensor(1.0), constraint=dist.constraints.positive)))
                pyro.sample("weights", dist.Normal(pyro.param("weights_loc", torch.zeros(n_features)),
                                                   pyro.param("weights_scale", torch.ones(n_features), constraint=dist.constraints.positive)).to_event(1))
                pyro.sample("sigma", dist.HalfNormal(pyro.param("sigma_loc", torch.tensor(1.0), constraint=dist.constraints.positive)))

            pyro.clear_param_store()
            svi = SVI(model, guide, Adam({"lr": 0.01}), loss=Trace_ELBO())

            for ep in range(n_epochs):
                svi.step(X_train_t, y_train_t)

            val_loss = svi.evaluate_loss(X_val_t, y_val_t)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_params = {
                    'scaler': scaler,
                    'weights': pyro.param("weights_loc").detach().numpy(),
                    'intercept': pyro.param("intercept_loc").item()
                }

        if best_model_params is None:
            print(f"Skipping outer fold {outer_fold} due to no valid inner models.")
            continue

        X_outer = outer_val[features].values
        y_outer_log = np.log1p(outer_val['Damage to Property'].values)
        scaler = best_model_params['scaler']
        X_outer_scaled = scaler.transform(X_outer)
        y_pred_log = best_model_params['intercept'] + X_outer_scaled @ best_model_params['weights']
        y_pred_orig = np.expm1(y_pred_log)
        y_true_orig = np.expm1(y_outer_log)

        mae_log, rmse_log = safe_metrics(y_outer_log, y_pred_log)
        mae_orig, rmse_orig = safe_metrics(y_true_orig, y_pred_orig)

        error_df = pd.DataFrame({
            'true_log': y_outer_log,
            'pred_log': y_pred_log,
            'true_orig': y_true_orig,
            'pred_orig': y_pred_orig,
            'Latitude': outer_val['Latitude'].values,
            'Longitude': outer_val['Longitude'].values,
            'outer_fold': outer_fold
        })
        #error_df['abs_error_log'] = np.abs(error_df['true_log'] - error_df['pred_log'])
        #error_df['abs_error_orig'] = np.abs(error_df['true_orig'] - error_df['pred_orig'])

        error_df['abs_error_log'] = np.abs(error_df['true_log'] - error_df['pred_log'])
        error_df['rmse_error_log'] = (error_df['true_log'] - error_df['pred_log']) ** 2
        error_df['abs_error_orig'] = np.abs(error_df['true_orig'] - error_df['pred_orig'])
        error_df['rmse_error_orig'] = (error_df['true_orig'] - error_df['pred_orig']) ** 2

        outer_results.append({
            'fold': outer_fold,
            'mae_log': mae_log,
            'rmse_log': rmse_log,
            'mae_orig': mae_orig,
            'rmse_orig': rmse_orig
        })

        all_error_dfs.append(error_df)

    results_df = pd.DataFrame(outer_results)
    full_error_df = pd.concat(all_error_dfs, ignore_index=True)
    return results_df, full_error_df


In [7]:
# Define your feature columns
features = [
    'log_pop', 'log_medinc',
    'pct_unemp', 'pct_limm', 'pct_noveh',
    'low_inc_pct', 'pop_poverty', 'high_inc_pct',
    'ppt_mean', 'tmean_mean'
]

# Run M6 model: Multistage Spatial Model with MLP-based inner clustering
results_df_M6, full_error_df_M6 = nested_spatial_cv_with_mlp(
    train_data=train_full,
    features=features,
    outer_k=8,
    inner_k=4,
    n_epochs=2000
)



=== OUTER Fold 1/8 ===

--- INNER Fold 1/4 ---

--- INNER Fold 2/4 ---
Skipping empty inner fold 1

--- INNER Fold 3/4 ---
Skipping empty inner fold 2

--- INNER Fold 4/4 ---

=== OUTER Fold 2/8 ===

--- INNER Fold 1/4 ---
Skipping empty inner fold 0

--- INNER Fold 2/4 ---

--- INNER Fold 3/4 ---

--- INNER Fold 4/4 ---

=== OUTER Fold 3/8 ===

--- INNER Fold 1/4 ---

--- INNER Fold 2/4 ---

--- INNER Fold 3/4 ---

--- INNER Fold 4/4 ---
Skipping empty inner fold 3

=== OUTER Fold 4/8 ===

--- INNER Fold 1/4 ---
Skipping empty inner fold 0

--- INNER Fold 2/4 ---

--- INNER Fold 3/4 ---
Skipping empty inner fold 2

--- INNER Fold 4/4 ---

=== OUTER Fold 5/8 ===

--- INNER Fold 1/4 ---

--- INNER Fold 2/4 ---

--- INNER Fold 3/4 ---

--- INNER Fold 4/4 ---
Skipping empty inner fold 3

=== OUTER Fold 6/8 ===

--- INNER Fold 1/4 ---
Skipping empty inner fold 0

--- INNER Fold 2/4 ---
Skipping empty inner fold 1

--- INNER Fold 3/4 ---
Skipping empty inner fold 2

--- INNER Fold 4/4 ---


In [8]:
import os

# Create the csv directory if it doesn't exist
os.makedirs('csv', exist_ok=True)

# Save the DataFrames
results_df_M6.to_csv('csv/results_df_M6.csv', index=True)
full_error_df_M6.to_csv('csv/full_error_df_M6.csv', index=True)


In [9]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def plot_combined_log_mae(error_df, output_file="M6_combined_log_mae.png"):
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([error_df['Longitude'].min() - 1, error_df['Longitude'].max() + 1,
                   error_df['Latitude'].min() - 1, error_df['Latitude'].max() + 1], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='black', linewidth=0.6)

    sc = ax.scatter(
        error_df['Longitude'], error_df['Latitude'],
        c=error_df['abs_error_log'], cmap='viridis', s=30, alpha=0.8,
        transform=ccrs.PlateCarree(), vmin=0, vmax=20
    )

    cbar = plt.colorbar(sc, ax=ax, orientation='vertical', shrink=0.8, pad=0.02)
    cbar.set_label("Log MAE", fontsize=12)
    cbar.set_ticks([0, 5, 10, 15, 20, 25])

    ax.set_title("Log MAE Error (All Outer Folds Combined)", fontsize=14)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.tight_layout()
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved combined error map: {output_file}")
plot_combined_log_mae(full_error_df_M6)

Saved combined error map: M6_combined_log_mae.png


In [10]:
# Define train loss and keep track of train loss in the nested_spatial_cv_with_mlp
# def plot_loss_curves(outer_results, output_prefix="fold"):
#     for result in outer_results:
#         train_losses = result['train_losses']
#         val_losses = result['val_losses']
#         fold = result['fold']

#         plt.figure(figsize=(8, 5))
#         plt.plot(train_losses, label='Train Loss', color='blue')
#         plt.plot(val_losses, label='Validation Loss', color='orange')
#         plt.xlabel("Epoch")
#         plt.ylabel("ELBO Loss")
#         plt.title(f"Train vs Validation Loss - Outer Fold {fold}")
#         plt.legend()
#         plt.grid(True)
#         plt.tight_layout()
#         fname = f"M6_{output_prefix}{fold}_loss_curve.png"
#         plt.savefig(fname, dpi=300)
#         plt.close()
#         print(f"Saved: {fname}")
        
# plot_loss_curves(results_df_M6.to_dict('records'))


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_pred_vs_obs(error_df, log=True, filename=None):
    if log:
        x = error_df['true_log']
        y = error_df['pred_log']
        scale = 'Log Scale'
        label = 'Log Damage'
    else:
        x = error_df['true_orig']
        y = error_df['pred_orig']
        scale = 'Original Scale'
        label = 'Damage to Property'

    # Scatterplot
    plt.figure(figsize=(7, 6))
    sns.scatterplot(x=x, y=y, s=20, alpha=0.6, edgecolor=None)
    
    # Line of perfect prediction
    min_val = min(x.min(), y.min())
    max_val = max(x.max(), y.max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=1.5, label="1:1 Line")

    # Labels and aesthetics
    plt.xlabel(f"Observed {label}")
    plt.ylabel(f"Predicted {label}")
    plt.title(f"Predicted vs. Observed ({scale})")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        print(f"Saved: {filename}")
        plt.close()
    else:
        plt.show()

# ──────────────── Call for both log and original scale ────────────────
plot_pred_vs_obs(full_error_df_M6, log=True, filename="M6_scatter_logspace.png")
plot_pred_vs_obs(full_error_df_M6, log=False, filename="M6_scatter_origspace.png")


Saved: M6_scatter_logspace.png
Saved: M6_scatter_origspace.png
